In [2]:
import numpy as np
import pandas as pd
import os

# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# from scipy.special import rel_entr
# from scipy.stats import entropy
# import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [3]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
! nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


# MongoDB

In [5]:
from pymongo import MongoClient
import pymongo

In [6]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2015 = db.rm_patent_cn_2015
    db_rm_us_2015 = db.rm_patent_us_2015
    db_rm_de_2015 = db.rm_patent_de_2015

    
except errors.ConnectionFailure as err:
    print(err)

# 2015

## Company數量

In [7]:
data_assignee = db_rm_de_2015.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [8]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_cn_2015.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
data_assignee = db_rm_us_2015.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [11]:
final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [12]:
# 取總數前30家公司
final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)

,company,count
734,Samsung Electronics Co Ltd,346
419,Intel Corp,268
872,Telefonaktiebolaget LM Ericsson AB,92
675,Qualcomm Inc,87
158,Chengdu Qinchuan IoT Technology Co Ltd,82
725,SUZHOU XUANHE IOT TECHNOLOGY Co Ltd,81
456,Jiangsu IoT Research and Development Center,61
192,Cisco Technology Inc,51
551,Microsoft Technology Licensing LLC,48
33,Afero Inc,48


In [13]:
com_df = final_df.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(30)
com_df.reset_index(drop=True, inplace=True)

In [14]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Samsung Electronics Co Ltd,346,5.846439
1,Intel Corp,268,5.590987
2,Telefonaktiebolaget LM Ericsson AB,92,4.521789
3,Qualcomm Inc,87,4.465908
4,Chengdu Qinchuan IoT Technology Co Ltd,82,4.406719
5,SUZHOU XUANHE IOT TECHNOLOGY Co Ltd,81,4.394449
6,Jiangsu IoT Research and Development Center,61,4.110874
7,Cisco Technology Inc,51,3.931826
8,Microsoft Technology Licensing LLC,48,3.871201
9,Afero Inc,48,3.871201


公司名稱正規化

In [15]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba"))

In [16]:
com_df

,company,count,count_log
0,Samsung Electronics,346,5.846439
1,Intel,268,5.590987
2,Telefonaktiebolaget LM Ericsson AB,92,4.521789
3,Qualcomm,87,4.465908
4,Chengdu Qinchuan IoT Technology,82,4.406719
5,SUZHOU XUANHE IOT TECHNOLOGY,81,4.394449
6,Jiangsu IoT Research and Development Center,61,4.110874
7,Cisco Technology,51,3.931826
8,Microsoft Technology Licensing LLC,48,3.871201
9,Afero,48,3.871201


In [17]:
com_ls = list(com_df.company)

# Data 2015

富比士2000資料

In [18]:
f_data = pd.read_csv("./Forbes/Forbes_2015.csv")

In [19]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Market Value,Revenue,Profits,Assets,Rank,Sector,Industry,Continent,Country,Headquarters,State,CEO,Forbes Webpage,Profits as % of Assets,Profits as % of Revenue
17,Samsung Electronics,199.356,195.891,21.9278,209.637,18,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Oh-Hyun Kwon,http://www.forbes.com/companies/samsung-electr...,0.104599,0.111939
66,Intel,147.197,55.870,11.7040,91.956,67,Information Technology,Semiconductors,North America,United States,California,California,Brian Krzanich,http://www.forbes.com/companies/intel/,0.127278,0.209486
155,Qualcomm,111.774,26.964,8.0630,48.447,156,Information Technology,Semiconductors,North America,United States,California,California,Steven Mollenkopf,http://www.forbes.com/companies/qualcomm/,0.166429,0.299028
157,Taiwan Semiconductor,123.323,25.173,8.7086,47.311,158,Information Technology,Semiconductors,Asia,Taiwan,Taiwan,NaN,Te Liu,http://www.forbes.com/companies/taiwan-semicon...,0.184071,0.345950
353,SK Hynix,29.994,16.269,3.9856,24.458,354,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Seong-Wook Park,http://www.forbes.com/companies/sk-hynix/,0.162957,0.244981
354,Micron Technology,29.551,16.948,3.8930,23.818,355,Information Technology,Semiconductors,North America,United States,Idaho,Idaho,D Durcan,http://www.forbes.com/companies/micron-technol...,0.163448,0.229703
417,Texas Instruments,59.488,13.045,2.7780,17.722,418,Information Technology,Semiconductors,North America,United States,Texas,Texas,Richard Templeton,http://www.forbes.com/companies/texas-instrume...,0.156754,0.212955
592,ASML Holding,44.830,7.768,1.5897,15.812,593,Information Technology,Semiconductors,Europe,Netherlands,Netherlands,NaN,Peter Wennink,http://www.forbes.com/companies/asml-holding/,0.100538,0.204647
681,Applied Materials,27.228,9.241,1.1670,13.073,682,Information Technology,Semiconductors,North America,United States,California,California,Gary Dickerson,http://www.forbes.com/companies/applied-materi...,0.089268,0.126285
787,Mediatek,20.947,7.031,1.5311,11.111,788,Information Technology,Semiconductors,Asia,Taiwan,Taiwan,NaN,Ming Tsai,http://www.forbes.com/companies/mediatek/,0.137800,0.217764


In [20]:
Ind_ls = list(f_data[f_data["Industry"]=="Semiconductors"].Company)

## 使用company top 30 做join

In [21]:
forb_ls = list(np.where(com_df.company.isin(Ind_ls), 1, 0))

In [22]:
com_df[com_df.company.isin(Ind_ls)]

,company,count,count_log
0,Samsung Electronics,346,5.846439
1,Intel,268,5.590987
3,Qualcomm,87,4.465908


In [25]:
# pbc 0.09, 0.09
comp_ls = ["Intel","Qualcomm","Telefonaktiebolaget LM Ericsson AB","Chengdu Qinchuan IoT Technology","Jiangsu IoT Research and Development Center","SUZHOU XUANHE IOT TECHNOLOGY"]

In [26]:
competitor_ls = list(np.where(com_df.company.isin(comp_ls), 1, 0))

In [27]:
com_df[com_df.company.isin(comp_ls)]

,company,count,count_log
1,Intel,268,5.590987
2,Telefonaktiebolaget LM Ericsson AB,92,4.521789
3,Qualcomm,87,4.465908
4,Chengdu Qinchuan IoT Technology,82,4.406719
5,SUZHOU XUANHE IOT TECHNOLOGY,81,4.394449
6,Jiangsu IoT Research and Development Center,61,4.110874


In [28]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.14346946548878017
ari:  0.2705943901163025


## 使用Forbes做join

In [29]:
forb_ls = list(np.where(f_data.Company.isin(Ind_ls), 1, 0))

In [30]:
# pbc 0.09, 0.09
comp_ls = ["Intel","Qualcomm","Telefonaktiebolaget LM Ericsson AB","Chengdu Qinchuan IoT Technology","Jiangsu IoT Research and Development Center","SUZHOU XUANHE IOT TECHNOLOGY"]

In [31]:
competitor_ls = list(np.where(f_data.Company.isin(comp_ls), 1, 0))

In [32]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.0926536567529152
ari:  0.11421330242159464
